In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Not sure where the unnamed column came from but just drop it
df = pd.read_csv('../data/processed_data.csv').drop(u'Unnamed: 0', axis=1) 
nan_indices = pd.isnull(df).any(1).nonzero()[0]
df = df.dropna()
print(df.shape)
print(df.columns)
df.head()

(25697, 13)
Index([u'season', u'Opponent DRtg/A', u'shot_distance', u'left_side',
       u'middle', u'right_side', u'moving', u'playoffs', u'home', u'segababa',
       u'Dunk or Layup', u'shot_made_flag', u'total_seconds_remaining'],
      dtype='object')


,season,Opponent DRtg/A,shot_distance,left_side,middle,right_side,moving,playoffs,home,segababa,Dunk or Layup,shot_made_flag,total_seconds_remaining
0,1,108.06,256.0,0,0,1,0,0,1,0,0,0.0,2202
1,1,101.68,228.0,0,0,1,0,0,0,0,0,0.0,2048
2,1,109.65,323.0,0,0,1,0,0,0,1,0,1.0,1957
3,1,109.65,0.0,0,1,0,0,0,0,1,0,0.0,1834
4,1,109.65,148.0,0,1,0,0,0,0,1,0,1.0,1767


In [11]:
df_submit = pd.read_csv('../data/submit_data.csv').drop(u'Unnamed: 0', axis=1)
print df_submit.shape
df_submit.head()

(5000, 14)


,shot_id,season,Opponent DRtg/A,shot_distance,left_side,middle,right_side,moving,playoffs,home,segababa,Dunk or Layup,shot_made_flag,total_seconds_remaining
0,22907,1,108.57,102.0,0,1,0,0,0,0,0,0,NaN,1878
1,22909,1,108.57,0.0,0,1,0,0,0,0,0,1,NaN,925
2,22927,1,109.44,256.0,0,0,1,0,0,0,0,0,NaN,1811
3,22926,1,109.44,15.0,1,0,0,0,0,0,0,0,NaN,1860
4,22930,1,109.44,250.0,0,1,0,0,0,0,0,0,NaN,1538


In [3]:
y = df['shot_made_flag']
X = df.drop('shot_made_flag', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## XGBoost

In [4]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}

clf = GridSearchCV(
    xgb.XGBClassifier(**ind_params),
    cv_params,
    scoring = 'neg_log_loss', 
    cv=3, n_jobs=-1
)


clf.fit(X_train, y_train)

print "Done" # Takes awhile to run...


Done


In [5]:
print clf.cv_results_

{'std_train_score': array([ 0.00199417,  0.00238134,  0.00231706,  0.00258923,  0.00263199,
        0.00049706,  0.00069928,  0.00107259,  0.00108461]), 'rank_test_score': array([3, 1, 2, 6, 5, 4, 9, 8, 7]), 'mean_score_time': array([ 0.17533334,  0.18666657,  0.14399997,  0.2603333 ,  0.17866659,
        0.24299995,  0.41000001,  0.3913335 ,  0.24733329]), 'param_max_depth': masked_array(data = [3 3 3 5 5 5 7 7 7],
             mask = [False False False False False False False False False],
       fill_value = ?)
, 'std_test_score': array([ 0.00394576,  0.0042392 ,  0.00476503,  0.00602323,  0.00725413,
        0.00593561,  0.01022372,  0.00821962,  0.00507168]), 'split1_train_score': array([-0.54558466, -0.55257965, -0.55803456, -0.36252589, -0.39651873,
       -0.4185962 , -0.17724569, -0.2388517 , -0.28167255]), 'split0_test_score': array([-0.67724567, -0.67567974, -0.67702904, -0.73557658, -0.73223314,
       -0.72319202, -0.83005716, -0.81047395, -0.7858409 ]), 'mean_test_score':

In [6]:
print clf.score(X_test, y_test)

C:\Anaconda2\lib\site-packages\sklearn\metrics\scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


-0.660103292689


## Create a submission

In [25]:
X_submit = df_submit.drop(['shot_id', 'shot_made_flag'], axis=1)
predictions = clf.predict(X_submit)
submission = pd.DataFrame()
submission["shot_id"] = df_submit['shot_id']
submission["shot_made_flag"] = predictions
submission = submission.sort_values(['shot_id'], ascending=True)
submission.head()

,shot_id,shot_made_flag
694,1,0.0
695,8,0.0
697,17,1.0
696,20,1.0
705,33,0.0


In [26]:
submission.shape

(5000, 2)

In [27]:
submission.to_csv("../data/submission_XGBoost.csv",index=False)